In [114]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [115]:
# File paths
file_paths = [
    "../data/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv",
    "../data/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv",
    "../data/Friday-WorkingHours-Morning.pcap_ISCX.csv",
    "../data/Monday-WorkingHours.pcap_ISCX.csv",
    "../data/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv",
    "../data/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv",
    "../data/Tuesday-WorkingHours.pcap_ISCX.csv",
    "../data/Wednesday-workingHours.pcap_ISCX.csv"
]

In [116]:
dataframes = []
for path in file_paths:
    try:
        df = pd.read_csv(path)
        dataframes.append(df)
    except Exception as e:
        print(f"Error loading {path}: {e}")

# Combine all data into a single dataframe
combined_df = pd.concat(dataframes, ignore_index=True)

In [117]:
# Drop any columns with all NaN values and drop rows with any NaN values
cleaned_df = combined_df.dropna(axis=1, how='all')  # Drop columns with all NaN
cleaned_df = cleaned_df.dropna()  # Drop rows with any NaN values

# Remove leading and trailing whitespaces from column names
cleaned_df.columns = cleaned_df.columns.str.strip()

In [118]:
# Separate features and labels
features = cleaned_df.drop(columns=['Label'])
labels = cleaned_df['Label']

In [119]:
# Replace infinite values with NaN and then drop rows containing NaN values
features.replace([np.inf, -np.inf], np.nan, inplace=True)
features.dropna(inplace=True)

# Update labels to match the cleaned features
labels = labels[features.index]

In [120]:
# Encode categorical labels into numerical values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

In [121]:
# Standardize the feature columns to have zero mean and unit variance
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [122]:
# Split data into training, validation, and test sets (60% train, 20% validation, 20% test)
X_train, X_temp, y_train, y_temp = train_test_split(scaled_features, encoded_labels, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [123]:
# Display the size of each dataset
print(f"Training set size: {X_train.shape}")
print(f"Validation set size: {X_val.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (1696725, 78)
Validation set size: (565575, 78)
Test set size: (565576, 78)


In [124]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_curve, auc, classification_report, confusion_matrix, log_loss, matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
import torch.nn.functional as F
import os
import json
import csv

In [125]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

results_dir = "../data/results/malicious_pattern"
os.makedirs(results_dir, exist_ok=True)

In [126]:
class IncrementalLearningModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(IncrementalLearningModel, self).__init__()
        # Define neural network structure with hidden layers
        self.fc1 = nn.Linear(input_size, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

    def train_model(self, X_train, y_train, X_val, y_val, num_epochs=50, lr=1e-4, poisoning_rate=0.0):
        # Convert numpy arrays to torch tensors
        train_data = X_train.clone().detach().float().to(device) if isinstance(X_train, torch.Tensor) else torch.tensor(X_train, dtype=torch.float32).to(device)
        train_labels = y_train.clone().detach().long().to(device) if isinstance(y_train, torch.Tensor) else torch.tensor(y_train, dtype=torch.long).to(device)
        val_data = X_val.clone().detach().float().to(device) if isinstance(X_val, torch.Tensor) else torch.tensor(X_val, dtype=torch.float32).to(device)
        val_labels = y_val.clone().detach().long().to(device) if isinstance(y_val, torch.Tensor) else torch.tensor(y_val, dtype=torch.long).to(device)

        # Define loss function and optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=lr)

        # Lists to store losses for visualization
        train_losses = []
        val_losses = []

        # Training loop
        for epoch in range(num_epochs):
            self.train()
            optimizer.zero_grad()
            outputs = self(train_data)
            loss = criterion(outputs, train_labels)
            loss.backward()
            optimizer.step()

            # Record training loss
            train_losses.append(loss.item())

            # Validation step
            self.eval()
            with torch.no_grad():
                val_outputs = self(val_data)
                val_loss = criterion(val_outputs, val_labels)
                val_losses.append(val_loss.item())

            print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

        # Create directory for the specific poisoning rate
        rate_dir = os.path.join(results_dir, f'poisoning_rate_{poisoning_rate:.2f}')
        os.makedirs(rate_dir, exist_ok=True)

        # Plot the loss curves and save
        plt.figure(figsize=(10, 5))
        plt.plot(train_losses, label='Training Loss')
        plt.plot(val_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title(f'Training and Validation Loss (Poisoning Rate {poisoning_rate:.2f})')
        plt.legend()
        plt.savefig(os.path.join(rate_dir, 'training_validation_loss.png'))
        plt.show()

    def evaluate(self, X_test, y_test, poisoning_rate):
        # Convert numpy arrays to torch tensors
        test_data = X_test.clone().detach().float().to(device) if isinstance(X_test, torch.Tensor) else torch.tensor(X_test, dtype=torch.float32).to(device)
        test_labels = y_test.clone().detach().long().to(device) if isinstance(y_test, torch.Tensor) else torch.tensor(y_test, dtype=torch.long).to(device)

        # Predict the labels for the test data
        self.eval()
        with torch.no_grad():
            outputs = self(test_data)
            _, predicted = torch.max(outputs, 1)

        # Calculate accuracy, precision, recall, and F1 score
        accuracy = accuracy_score(test_labels.cpu().numpy(), predicted.cpu().numpy())
        precision = precision_score(test_labels.cpu().numpy(), predicted.cpu().numpy(), average='weighted', zero_division=1)
        recall = recall_score(test_labels.cpu().numpy(), predicted.cpu().numpy(), average='weighted', zero_division=1)
        f1 = f1_score(test_labels.cpu().numpy(), predicted.cpu().numpy(), average='weighted', zero_division=1)
        
        print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")
        
        # Generate classification report
        print("\nClassification Report:")
        print(classification_report(test_labels.cpu().numpy(), predicted.cpu().numpy(), zero_division=1))

        # Create subdirectory for each poisoning rate
        rate_dir = os.path.join(results_dir, f'poisoning_rate_{poisoning_rate:.2f}')
        os.makedirs(rate_dir, exist_ok=True)

        # Plot Confusion Matrix
        cm = confusion_matrix(test_labels.cpu().numpy(), predicted.cpu().numpy())
        plt.figure(figsize=(10, 8))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'Confusion Matrix (Poisoning Rate {poisoning_rate:.2f})')
        plt.savefig(os.path.join(rate_dir, 'confusion_matrix.png'))
        plt.show()

        # Generate ROC Curve and AUC if binary classification
        if len(np.unique(test_labels.cpu().numpy())) == 2:  # Only applicable for binary classification
            fpr, tpr, _ = roc_curve(test_labels.cpu().numpy(), outputs[:, 1].cpu().numpy())
            roc_auc = auc(fpr, tpr)
            plt.figure()
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'Receiver Operating Characteristic (ROC) Curve (Poisoning Rate {poisoning_rate:.2f})')
            plt.legend(loc="lower right")
            plt.savefig(os.path.join(rate_dir, 'roc_curve.png'))
            plt.show()

        return accuracy, precision, recall, f1

In [127]:
# Calculate feature importance
def calculate_feature_importance(X_train, y_train):
    # Using a simple RandomForest to evaluate feature importance
    rf = RandomForestClassifier(n_estimators=50, random_state=42)
    rf.fit(X_train, y_train)
    result = permutation_importance(rf, X_train, y_train, n_repeats=10, random_state=42)
    return result.importances_mean

In [128]:
def poison_data(data, labels, poisoning_rate=0.1, strategy='pgd_attack', model=None, criterion=None, epsilon=0.1, alpha=0.01, iters=10):
    """
    Generate poisoned data using different strategies.

    Parameters:
    - data: np.array or torch.Tensor, original input data
    - labels: np.array or torch.Tensor, original labels
    - poisoning_rate: float, proportion of data to be poisoned
    - strategy: str, the type of poisoning strategy ('label_flip', 'feature_perturbation', 'pgd_attack', 'logic_disruption', 'malicious_pattern', 'combined')
    - model: (optional) the model used for generating adversarial perturbation (required for 'pgd_attack')
    - criterion: (optional) loss function used for generating adversarial perturbation (required for 'pgd_attack')
    - epsilon: float, perturbation strength for adversarial attacks
    - alpha: float, step size for PGD attack
    - iters: int, number of iterations for PGD attack

    Returns:
    - poisoned_data: torch.Tensor, data with poisoning applied
    - poisoned_labels: torch.Tensor, labels with poisoning applied
    """
    data = data.clone().detach().float().to(device) if isinstance(data, torch.Tensor) else torch.tensor(data, dtype=torch.float32).to(device)
    labels = labels.clone().detach().long().to(device) if isinstance(labels, torch.Tensor) else torch.tensor(labels, dtype=torch.long).to(device)

    num_samples = data.shape[0]
    num_poisoned = int(poisoning_rate * num_samples)
    poisoned_indices = torch.tensor(random.sample(range(num_samples), num_poisoned), dtype=torch.long).to(device)

    if strategy == 'pgd_attack':
        # PGD Attack
        if model is None or criterion is None:
            raise ValueError("Model and criterion are required for 'pgd_attack' strategy.")
        
        poisoned_data = data.clone().detach().to(device)
        poisoned_data.requires_grad = False

        batch_size = 16
        for i in range(iters):
            for start_idx in range(0, num_poisoned, batch_size):
                end_idx = min(start_idx + batch_size, num_poisoned)
                batch_indices = poisoned_indices[start_idx:end_idx]
                batch_data = poisoned_data[batch_indices].clone().detach().requires_grad_(True)  # Ensure batch_data is a leaf tensor
                batch_labels = labels[batch_indices]

                # Forward pass through the model
                outputs = model(batch_data)

                # Compute loss
                loss = criterion(outputs, batch_labels)

                # Zero out any previous gradients
                model.zero_grad()
                if batch_data.grad is not None:
                    batch_data.grad.zero_()

                # Backward pass
                loss.backward()

                # Retain gradients on batch_data so we can use them
                batch_data.retain_grad()

                # Check if gradients are available
                if batch_data.grad is not None:
                    # Compute perturbation
                    gradient = batch_data.grad.sign()
                    batch_data = batch_data + alpha * gradient

                    # Clamp the perturbed data
                    batch_data = torch.clamp(batch_data, min=data[batch_indices] - epsilon, max=data[batch_indices] + epsilon)
                    batch_data = torch.clamp(batch_data, 0, 1)  # Assuming the data is in range [0, 1]

                    # Update the poisoned data (avoid in-place modification of leaf variable)
                    poisoned_data = poisoned_data.clone().detach()
                    poisoned_data[batch_indices] = batch_data.clone().detach()  # Use clone().detach() to avoid in-place update of tensor requiring grad
                else:
                    print("Warning: Gradient is None during PGD attack.")

            torch.cuda.empty_cache()

        return poisoned_data.detach(), labels

    elif strategy == 'label_flip':
        # Vectorized label flipping
        poisoned_labels = labels.clone()
        poisoned_labels[poisoned_indices] = (labels[poisoned_indices] + 1) % len(torch.unique(labels))
        return data, poisoned_labels

    elif strategy == 'feature_perturbation':
        # Vectorized feature perturbation
        poisoned_data = data.clone()
        perturbation = torch.randn_like(poisoned_data[poisoned_indices]) * 1.0  # Larger perturbation scale
        poisoned_data[poisoned_indices] += perturbation
        return poisoned_data, labels

    elif strategy == 'combined':
        # Combined label flipping and feature perturbation
        poisoned_data = data.clone()
        poisoned_labels = labels.clone()

        # Apply feature perturbation
        perturbation = torch.randn_like(poisoned_data[poisoned_indices]) * 0.5
        poisoned_data[poisoned_indices] += perturbation

        # Flip labels
        poisoned_labels[poisoned_indices] = (labels[poisoned_indices] + 1) % len(torch.unique(labels))

        return poisoned_data, poisoned_labels

    elif strategy == 'logic_disruption':
        # Logic Disruption (breaking feature relationships)
        poisoned_data = data.clone()
        for feature_idx in [0, 1, 2]:
            poisoned_data[:, feature_idx] = poisoned_data[:, feature_idx][torch.randperm(poisoned_data.size(0))]
        return poisoned_data, labels

    elif strategy == 'malicious_pattern':
        # Malicious Pattern Injection
        poisoned_data = data.clone()
        pattern_strength = 0.5
        pattern = torch.randn(len(poisoned_indices), data.size(1)).to(device) * pattern_strength
        poisoned_data[poisoned_indices] += pattern

        # Randomly flip some labels to further confuse the model
        poisoned_labels = labels.clone()
        poisoned_labels[poisoned_indices] = (labels[poisoned_indices] + 1) % len(torch.unique(labels))

        return poisoned_data, poisoned_labels

    else:
        raise ValueError("Unsupported poisoning strategy. Available strategies: 'label_flip', 'feature_perturbation', 'pgd_attack', 'logic_disruption', 'malicious_pattern', 'combined'.")


In [129]:
# Function to handle catastrophic forgetting using Replay mechanism
def replay_mechanism(model, previous_data, new_data, previous_labels, new_labels, num_epochs=5, lr=1e-4):
    # Convert numpy arrays to torch tensors
    previous_data = previous_data.clone().detach().float().to(device) if isinstance(previous_data, torch.Tensor) else torch.tensor(previous_data, dtype=torch.float32).to(device)
    previous_labels = previous_labels.clone().detach().long().to(device) if isinstance(previous_labels, torch.Tensor) else torch.tensor(previous_labels, dtype=torch.long).to(device)
    new_data = new_data.clone().detach().float().to(device) if isinstance(new_data, torch.Tensor) else torch.tensor(new_data, dtype=torch.float32).to(device)
    new_labels = new_labels.clone().detach().long().to(device) if isinstance(new_labels, torch.Tensor) else torch.tensor(new_labels, dtype=torch.long).to(device)

    # Combine previous and new data
    combined_data = torch.cat((previous_data, new_data), dim=0)
    combined_labels = torch.cat((previous_labels, new_labels), dim=0)

    # Retrain the model with combined dataset
    model.train_model(combined_data, combined_labels, combined_data, combined_labels, num_epochs, lr)

In [130]:
def visualize_replay_impact(results):
    poisoning_rates = [result["poisoning_rate"] for result in results]
    accuracy_before = [result["accuracy_before"] for result in results]
    accuracy_after = [result["accuracy_after"] for result in results]
    f1_before = [result["f1_score_before"] for result in results]
    f1_after = [result["f1_score_after"] for result in results]

    # Plot Accuracy before and after Replay
    plt.figure(figsize=(10, 5))
    width = 0.3
    x = np.arange(len(poisoning_rates))
    plt.bar(x - width / 2, accuracy_before, width, label='Before Replay', color='b')
    plt.bar(x + width / 2, accuracy_after, width, label='After Replay', color='g')
    plt.xlabel('Poisoning Rate')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Before and After Replay')
    plt.xticks(x, [f"{rate:.2f}" for rate in poisoning_rates])
    plt.ylim(0, 1)
    plt.legend()
    plt.savefig(os.path.join(results_dir, 'accuracy_comparison.png'))
    plt.show()

    # Plot F1 Score before and after Replay
    plt.figure(figsize=(10, 5))
    plt.bar(x - width / 2, f1_before, width, label='Before Replay', color='r')
    plt.bar(x + width / 2, f1_after, width, label='After Replay', color='g')
    plt.xlabel('Poisoning Rate')
    plt.ylabel('F1 Score')
    plt.title('F1 Score Before and After Replay')
    plt.xticks(x, [f"{rate:.2f}" for rate in poisoning_rates])
    plt.ylim(0, 1)
    plt.legend()
    plt.savefig(os.path.join(results_dir, 'f1_score_comparison.png'))
    plt.show()

In [131]:
def run_poisoning_experiment_with_replay(model, X_train, y_train, X_val, y_val, X_test, y_test, poisoning_rates, replay_data, replay_labels, num_epochs=50, lr=1e-4):
    results = []
    results_csv_path = os.path.join(results_dir, "poisoning_experiment_results.csv")
    results_json_path = os.path.join(results_dir, "poisoning_experiment_results.json")

    with open(results_csv_path, mode='w', newline='', encoding='utf-8') as csv_file:
        csv_writer = csv.DictWriter(csv_file, fieldnames=[
            "poisoning_rate", "accuracy_before", "precision_before", "recall_before", "f1_score_before",
            "accuracy_after", "precision_after", "recall_after", "f1_score_after"
        ])
        csv_writer.writeheader()

        for poisoning_rate in poisoning_rates:
            print(f"\nRunning experiment with poisoning rate: {poisoning_rate:.2f}")

            # Initialize a new model for each experiment
            model_instance = IncrementalLearningModel(input_size=X_train.shape[1], num_classes=len(np.unique(y_train))).to(device)

            # Poison data
            poisoned_data, poisoned_labels = poison_data(X_train, y_train, poisoning_rate=poisoning_rate, strategy='malicious_pattern', model=model_instance, criterion=nn.CrossEntropyLoss())

            # Train model
            model_instance.train_model(poisoned_data, poisoned_labels, X_val, y_val, num_epochs=num_epochs, lr=lr, poisoning_rate=poisoning_rate)

            # Evaluate model before replay mechanism
            print("\nEvaluating model before replay...")
            accuracy_before, precision_before, recall_before, f1_before = model_instance.evaluate(X_test, y_test, poisoning_rate)

            # Apply replay mechanism
            print("\nApplying replay mechanism...")
            replay_mechanism(model_instance, replay_data, poisoned_data[:50], replay_labels, poisoned_labels[:50], num_epochs=5, lr=lr)

            # Evaluate model after replay mechanism
            print("\nEvaluating model after replay...")
            accuracy_after, precision_after, recall_after, f1_after = model_instance.evaluate(X_test, y_test, poisoning_rate)

            # Store results
            result = {
                "poisoning_rate": poisoning_rate,
                "accuracy_before": accuracy_before,
                "precision_before": precision_before,
                "recall_before": recall_before,
                "f1_score_before": f1_before,
                "accuracy_after": accuracy_after,
                "precision_after": precision_after,
                "recall_after": recall_after,
                "f1_score_after": f1_after
            }
            results.append(result)

            # Write results to CSV
            csv_writer.writerow(result)

    # Save results as JSON
    with open(results_json_path, mode='w', encoding='utf-8') as json_file:
        json.dump(results, json_file, indent=4)

    # Visualize results
    visualize_replay_impact(results)

In [ ]:
# Extract replay data from training data
replay_data, _, replay_labels, _ = train_test_split(X_train, y_train, test_size=0.9, random_state=42)

# Define different levels of poisoning rates to evaluate
poisoning_rates = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]  # Poisoning rates

# Initialize the model
model = IncrementalLearningModel(input_size=X_train.shape[1], num_classes=len(np.unique(y_train))).to(device)

# Run the poisoning experiments
run_poisoning_experiment_with_replay(
    model, X_train, y_train, X_val, y_val, X_test, y_test, poisoning_rates, replay_data, replay_labels
)